# Time Series Databases: InfluxDB & TimescaleDB

Time series databases (TSDBs) are specialized database systems optimized for storing, querying, and analyzing time-stamped data. Unlike traditional relational databases, TSDBs are designed to handle the unique characteristics and challenges of time series workloads.

## What is Time Series Data?

Time series data consists of sequences of data points indexed by time. Each data point typically includes:
- **Timestamp**: When the measurement was taken
- **Metrics/Values**: Numerical measurements (e.g., temperature, CPU usage, stock price)
- **Tags/Labels**: Metadata for categorization (e.g., sensor_id, region, host)

## Key Characteristics of Time Series Data

| Characteristic | Description |
|----------------|-------------|
| **Time-ordered** | Data arrives in chronological sequence |
| **Append-only** | New data is typically inserted, rarely updated |
| **High volume** | Millions of data points per second |
| **Recent data focus** | Most queries access recent data |
| **Aggregation-heavy** | Common operations: averages, sums, percentiles over time windows |
| **Downsampling** | Older data often stored at lower resolution |

## InfluxDB vs TimescaleDB

### InfluxDB
- Purpose-built time series database
- Custom query language (Flux) and InfluxQL
- Schema-less design
- Built-in retention policies and continuous queries
- Excellent for metrics and IoT data

### TimescaleDB
- PostgreSQL extension for time series
- Full SQL support with time series optimizations
- Hypertables for automatic partitioning
- Compression and continuous aggregates
- Best when you need relational + time series capabilities

```
┌─────────────────────────────────────────────────────────────┐
│                    Time Series Database                      │
├─────────────────────────────────────────────────────────────┤
│  ┌─────────────┐    ┌─────────────┐    ┌─────────────┐      │
│  │  InfluxDB   │    │ TimescaleDB │    │   Others    │      │
│  │             │    │             │    │ (QuestDB,   │      │
│  │  - Flux     │    │  - SQL      │    │  Prometheus)│      │
│  │  - Schema-  │    │  - Postgres │    │             │      │
│  │    less     │    │    based    │    │             │      │
│  └─────────────┘    └─────────────┘    └─────────────┘      │
└─────────────────────────────────────────────────────────────┘
```

## Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

## Generating Time Series Data

Let's simulate realistic time series data representing different use cases.

In [ ]:
def generate_sensor_data(start_date, periods, freq='1min', num_sensors=3):
    """
    Generate simulated IoT sensor data with realistic patterns.
    
    Parameters:
    -----------
    start_date : str
        Start date for the time series
    periods : int
        Number of data points to generate
    freq : str
        Frequency of data points (e.g., '1min', '5min', '1H')
    num_sensors : int
        Number of sensors to simulate
    
    Returns:
    --------
    pd.DataFrame
        DataFrame with timestamp, sensor_id, temperature, humidity, pressure
    """
    dates = pd.date_range(start=start_date, periods=periods, freq=freq)
    
    data = []
    for sensor_id in range(1, num_sensors + 1):
        # Base values with sensor-specific offsets
        base_temp = 20 + sensor_id * 2
        base_humidity = 50 + sensor_id * 5
        base_pressure = 1013 + sensor_id * 0.5
        
        for i, date in enumerate(dates):
            # Add daily seasonality (temperature higher during day)
            hour = date.hour
            daily_factor = np.sin(2 * np.pi * (hour - 6) / 24) * 5
            
            # Add noise and occasional spikes
            noise = np.random.normal(0, 0.5)
            spike = 10 if np.random.random() < 0.01 else 0
            
            data.append({
                'timestamp': date,
                'sensor_id': f'sensor_{sensor_id}',
                'temperature': base_temp + daily_factor + noise + spike,
                'humidity': base_humidity + noise * 2 + np.random.normal(0, 3),
                'pressure': base_pressure + np.random.normal(0, 0.2)
            })
    
    return pd.DataFrame(data)

# Generate 24 hours of sensor data at 1-minute intervals
sensor_df = generate_sensor_data('2024-01-15', periods=1440, freq='1min', num_sensors=3)
print(f"Generated {len(sensor_df):,} data points")
print(f"Date range: {sensor_df['timestamp'].min()} to {sensor_df['timestamp'].max()}")
sensor_df.head(10)

In [ ]:
def generate_financial_data(start_date, periods, freq='1min'):
    """
    Generate simulated stock price data with OHLCV format.
    
    Parameters:
    -----------
    start_date : str
        Start date for the time series
    periods : int
        Number of data points to generate
    freq : str
        Frequency of data points
    
    Returns:
    --------
    pd.DataFrame
        DataFrame with OHLCV data
    """
    dates = pd.date_range(start=start_date, periods=periods, freq=freq)
    
    # Generate random walk for price
    returns = np.random.normal(0.0001, 0.002, periods)
    price = 100 * np.cumprod(1 + returns)
    
    # Generate OHLC from the price
    data = {
        'timestamp': dates,
        'symbol': 'TECH',
        'open': price,
        'high': price * (1 + np.abs(np.random.normal(0, 0.001, periods))),
        'low': price * (1 - np.abs(np.random.normal(0, 0.001, periods))),
        'close': price * (1 + np.random.normal(0, 0.0005, periods)),
        'volume': np.random.randint(1000, 100000, periods)
    }
    
    return pd.DataFrame(data)

# Generate financial tick data
financial_df = generate_financial_data('2024-01-15 09:30:00', periods=390, freq='1min')
print(f"Generated {len(financial_df):,} OHLCV records")
financial_df.head(10)

In [ ]:
def generate_server_metrics(start_date, periods, freq='10s', num_hosts=5):
    """
    Generate simulated server monitoring metrics.
    
    Parameters:
    -----------
    start_date : str
        Start date for the time series
    periods : int
        Number of data points to generate
    freq : str
        Frequency of data points
    num_hosts : int
        Number of hosts to simulate
    
    Returns:
    --------
    pd.DataFrame
        DataFrame with server metrics
    """
    dates = pd.date_range(start=start_date, periods=periods, freq=freq)
    
    data = []
    for host_id in range(1, num_hosts + 1):
        # Simulate different baseline loads for different hosts
        base_cpu = 20 + host_id * 10
        base_memory = 40 + host_id * 8
        
        for i, date in enumerate(dates):
            # Simulate load spikes during business hours
            hour = date.hour
            business_hours_factor = 1.5 if 9 <= hour <= 17 else 1.0
            
            # Random spikes to simulate incidents
            incident = np.random.random() < 0.005
            incident_factor = 2.0 if incident else 1.0
            
            cpu = min(100, base_cpu * business_hours_factor * incident_factor + np.random.normal(0, 5))
            memory = min(100, base_memory + np.random.normal(0, 3))
            
            data.append({
                'timestamp': date,
                'host': f'web-server-{host_id:02d}',
                'region': ['us-east', 'us-west', 'eu-west', 'ap-south', 'ap-east'][host_id - 1],
                'cpu_percent': max(0, cpu),
                'memory_percent': max(0, memory),
                'disk_io_mbps': max(0, np.random.exponential(50)),
                'network_mbps': max(0, np.random.exponential(100)),
                'request_count': max(0, int(np.random.poisson(100 * business_hours_factor)))
            })
    
    return pd.DataFrame(data)

# Generate 1 hour of server metrics at 10-second intervals
server_df = generate_server_metrics('2024-01-15 10:00:00', periods=360, freq='10s', num_hosts=5)
print(f"Generated {len(server_df):,} server metric records")
server_df.head(10)

## Visualizing Time Series Data with Plotly

In [ ]:
# Visualize IoT sensor data
fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    subplot_titles=('Temperature (°C)', 'Humidity (%)', 'Pressure (hPa)'),
    vertical_spacing=0.08
)

colors = px.colors.qualitative.Set1

for i, sensor in enumerate(sensor_df['sensor_id'].unique()):
    sensor_data = sensor_df[sensor_df['sensor_id'] == sensor]
    
    fig.add_trace(
        go.Scatter(x=sensor_data['timestamp'], y=sensor_data['temperature'],
                   name=sensor, line=dict(color=colors[i]), legendgroup=sensor),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=sensor_data['timestamp'], y=sensor_data['humidity'],
                   name=sensor, line=dict(color=colors[i]), legendgroup=sensor,
                   showlegend=False),
        row=2, col=1
    )
    fig.add_trace(
        go.Scatter(x=sensor_data['timestamp'], y=sensor_data['pressure'],
                   name=sensor, line=dict(color=colors[i]), legendgroup=sensor,
                   showlegend=False),
        row=3, col=1
    )

fig.update_layout(
    title='IoT Sensor Data - 24 Hour Overview',
    height=700,
    template='plotly_white',
    hovermode='x unified'
)

fig

In [ ]:
# Visualize financial data with candlestick chart
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    row_heights=[0.7, 0.3],
    subplot_titles=('Price (Candlestick)', 'Volume'),
    vertical_spacing=0.08
)

# Candlestick chart
fig.add_trace(
    go.Candlestick(
        x=financial_df['timestamp'],
        open=financial_df['open'],
        high=financial_df['high'],
        low=financial_df['low'],
        close=financial_df['close'],
        name='TECH'
    ),
    row=1, col=1
)

# Volume bar chart
colors_volume = ['red' if close < open else 'green' 
                 for close, open in zip(financial_df['close'], financial_df['open'])]

fig.add_trace(
    go.Bar(
        x=financial_df['timestamp'],
        y=financial_df['volume'],
        marker_color=colors_volume,
        name='Volume'
    ),
    row=2, col=1
)

fig.update_layout(
    title='Financial Time Series - OHLCV Data',
    height=600,
    template='plotly_white',
    xaxis_rangeslider_visible=False,
    showlegend=False
)

fig

In [ ]:
# Visualize server metrics with heatmap for CPU across hosts
server_pivot = server_df.pivot_table(
    index='host', 
    columns='timestamp', 
    values='cpu_percent'
)

fig = go.Figure(data=go.Heatmap(
    z=server_pivot.values,
    x=server_pivot.columns,
    y=server_pivot.index,
    colorscale='RdYlGn_r',
    colorbar=dict(title='CPU %')
))

fig.update_layout(
    title='Server CPU Utilization Heatmap',
    xaxis_title='Time',
    yaxis_title='Host',
    height=400,
    template='plotly_white'
)

fig

## Retention Policies & Data Lifecycle Management

Retention policies define how long data is stored before automatic deletion. This is crucial for:
- **Storage cost management**: Old high-resolution data consumes space
- **Compliance**: Some data must be retained for specific periods
- **Performance**: Smaller datasets query faster

### InfluxDB Retention Policies

```sql
-- Create a retention policy in InfluxDB
CREATE RETENTION POLICY "30_days" ON "metrics_db" 
  DURATION 30d 
  REPLICATION 1 
  DEFAULT;

-- Create policy for long-term aggregated data
CREATE RETENTION POLICY "1_year" ON "metrics_db" 
  DURATION 365d 
  REPLICATION 1;
```

### TimescaleDB Data Retention

```sql
-- Add automatic data retention policy
SELECT add_retention_policy('sensor_data', INTERVAL '30 days');

-- Drop chunks older than 7 days manually
SELECT drop_chunks('sensor_data', older_than => INTERVAL '7 days');
```

In [ ]:
def apply_retention_policy(df, timestamp_col, retention_days):
    """
    Simulate applying a retention policy to time series data.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input DataFrame with time series data
    timestamp_col : str
        Name of the timestamp column
    retention_days : int
        Number of days to retain data
    
    Returns:
    --------
    pd.DataFrame
        DataFrame with old data removed
    """
    cutoff_date = df[timestamp_col].max() - timedelta(days=retention_days)
    retained_df = df[df[timestamp_col] > cutoff_date].copy()
    
    removed_count = len(df) - len(retained_df)
    print(f"Retention policy: {retention_days} days")
    print(f"Cutoff date: {cutoff_date}")
    print(f"Records removed: {removed_count:,}")
    print(f"Records retained: {len(retained_df):,}")
    
    return retained_df

# Example: Apply 12-hour retention (simulated)
retained_sensor_df = apply_retention_policy(
    sensor_df, 'timestamp', retention_days=0.5  # 12 hours
)

## Downsampling & Aggregation

Downsampling reduces data resolution by aggregating fine-grained data into coarser time buckets. This is essential for:
- **Long-term storage**: Store 1-minute data for 7 days, hourly data for 1 year
- **Query performance**: Aggregated data is faster to query
- **Visualization**: Minute-level data isn't useful for yearly trends

### Common Aggregation Functions
- `mean()`: Average value in time window
- `min()`/`max()`: Extremes in time window
- `sum()`: Total in time window
- `count()`: Number of data points
- `first()`/`last()`: First/last value in window

In [ ]:
def downsample_time_series(df, timestamp_col, group_cols, value_cols, freq, agg_funcs=None):
    """
    Downsample time series data to a lower resolution.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input DataFrame with time series data
    timestamp_col : str
        Name of the timestamp column
    group_cols : list
        Columns to group by (e.g., sensor_id, host)
    value_cols : list
        Columns to aggregate
    freq : str
        Target frequency (e.g., '5min', '1H', '1D')
    agg_funcs : dict
        Aggregation functions per column
    
    Returns:
    --------
    pd.DataFrame
        Downsampled DataFrame
    """
    if agg_funcs is None:
        agg_funcs = {col: ['mean', 'min', 'max'] for col in value_cols}
    
    df_copy = df.copy()
    df_copy['time_bucket'] = df_copy[timestamp_col].dt.floor(freq)
    
    grouped = df_copy.groupby(['time_bucket'] + group_cols)
    
    result = grouped[value_cols].agg(agg_funcs)
    result.columns = ['_'.join(col).strip() for col in result.columns.values]
    result = result.reset_index()
    
    return result

# Downsample sensor data from 1-minute to 15-minute intervals
downsampled_sensor = downsample_time_series(
    sensor_df,
    timestamp_col='timestamp',
    group_cols=['sensor_id'],
    value_cols=['temperature', 'humidity', 'pressure'],
    freq='15min'
)

print(f"Original records: {len(sensor_df):,}")
print(f"Downsampled records: {len(downsampled_sensor):,}")
print(f"Compression ratio: {len(sensor_df) / len(downsampled_sensor):.1f}x")
downsampled_sensor.head(10)

In [ ]:
# Visualize original vs downsampled data
sensor1_original = sensor_df[sensor_df['sensor_id'] == 'sensor_1']
sensor1_downsampled = downsampled_sensor[downsampled_sensor['sensor_id'] == 'sensor_1']

fig = go.Figure()

# Original data
fig.add_trace(go.Scatter(
    x=sensor1_original['timestamp'],
    y=sensor1_original['temperature'],
    name='Original (1-min)',
    line=dict(color='lightblue', width=1),
    opacity=0.7
))

# Downsampled mean
fig.add_trace(go.Scatter(
    x=sensor1_downsampled['time_bucket'],
    y=sensor1_downsampled['temperature_mean'],
    name='Downsampled Mean (15-min)',
    line=dict(color='blue', width=2)
))

# Min/Max range
fig.add_trace(go.Scatter(
    x=pd.concat([sensor1_downsampled['time_bucket'], sensor1_downsampled['time_bucket'][::-1]]),
    y=pd.concat([sensor1_downsampled['temperature_max'], sensor1_downsampled['temperature_min'][::-1]]),
    fill='toself',
    fillcolor='rgba(0,100,80,0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    name='Min-Max Range'
))

fig.update_layout(
    title='Original vs Downsampled Temperature Data (Sensor 1)',
    xaxis_title='Time',
    yaxis_title='Temperature (°C)',
    template='plotly_white',
    height=450
)

fig

## Compression Strategies

Time series databases use specialized compression algorithms optimized for time-ordered data:

### Timestamp Compression
- **Delta encoding**: Store differences between consecutive timestamps
- **Delta-of-delta**: Store differences of differences (excellent for regular intervals)

### Value Compression
- **Gorilla compression**: XOR-based compression for floating-point values
- **Run-length encoding (RLE)**: Compress repeated values
- **Dictionary encoding**: Replace repeated strings with integer codes

### TimescaleDB Compression

```sql
-- Enable compression on a hypertable
ALTER TABLE sensor_data SET (
    timescaledb.compress,
    timescaledb.compress_segmentby = 'sensor_id',
    timescaledb.compress_orderby = 'timestamp DESC'
);

-- Add automatic compression policy
SELECT add_compression_policy('sensor_data', INTERVAL '7 days');
```

In [ ]:
def analyze_compression_potential(df, timestamp_col, value_cols):
    """
    Analyze the compression potential of time series data.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input DataFrame
    timestamp_col : str
        Name of the timestamp column
    value_cols : list
        Numeric columns to analyze
    
    Returns:
    --------
    dict
        Compression analysis results
    """
    results = {}
    
    # Timestamp analysis - delta encoding potential
    timestamps = pd.to_datetime(df[timestamp_col]).astype(int) // 10**9  # Convert to seconds
    deltas = np.diff(timestamps)
    unique_deltas = len(np.unique(deltas))
    
    results['timestamp_analysis'] = {
        'total_timestamps': len(timestamps),
        'unique_deltas': unique_deltas,
        'delta_encoding_ratio': len(timestamps) / max(unique_deltas, 1),
        'most_common_delta_seconds': int(np.median(deltas)) if len(deltas) > 0 else 0
    }
    
    # Value analysis
    results['value_analysis'] = {}
    for col in value_cols:
        values = df[col].values
        
        # Calculate value deltas
        value_deltas = np.diff(values)
        
        # Estimate Gorilla compression potential (smaller deltas = better compression)
        mean_delta = np.mean(np.abs(value_deltas))
        std_values = np.std(values)
        
        results['value_analysis'][col] = {
            'mean_value': float(np.mean(values)),
            'std_value': float(std_values),
            'mean_delta': float(mean_delta),
            'delta_std_ratio': float(mean_delta / std_values) if std_values > 0 else 0,
            'unique_values': len(np.unique(values.round(2))),
            'total_values': len(values)
        }
    
    return results

# Analyze compression potential for sensor data
compression_analysis = analyze_compression_potential(
    sensor_df[sensor_df['sensor_id'] == 'sensor_1'],
    'timestamp',
    ['temperature', 'humidity', 'pressure']
)

print("=== Compression Analysis ===")
print("\nTimestamp Analysis:")
for key, value in compression_analysis['timestamp_analysis'].items():
    print(f"  {key}: {value}")

print("\nValue Column Analysis:")
for col, metrics in compression_analysis['value_analysis'].items():
    print(f"\n  {col}:")
    for key, value in metrics.items():
        print(f"    {key}: {value:.4f}" if isinstance(value, float) else f"    {key}: {value}")

## Continuous Aggregates (Materialized Views)

Continuous aggregates automatically maintain pre-computed aggregations that update incrementally as new data arrives.

### TimescaleDB Continuous Aggregates

```sql
-- Create a continuous aggregate for hourly averages
CREATE MATERIALIZED VIEW sensor_hourly
WITH (timescaledb.continuous) AS
SELECT 
    time_bucket('1 hour', timestamp) AS bucket,
    sensor_id,
    AVG(temperature) AS avg_temp,
    MIN(temperature) AS min_temp,
    MAX(temperature) AS max_temp,
    COUNT(*) AS sample_count
FROM sensor_data
GROUP BY bucket, sensor_id;

-- Add refresh policy
SELECT add_continuous_aggregate_policy('sensor_hourly',
    start_offset => INTERVAL '3 hours',
    end_offset => INTERVAL '1 hour',
    schedule_interval => INTERVAL '1 hour');
```

### InfluxDB Continuous Queries

```sql
-- Create a continuous query for downsampling
CREATE CONTINUOUS QUERY "cq_hourly_temp" ON "metrics_db"
BEGIN
    SELECT mean("temperature") AS "avg_temp",
           min("temperature") AS "min_temp",
           max("temperature") AS "max_temp"
    INTO "metrics_db"."1_year"."hourly_temperature"
    FROM "sensor_data"
    GROUP BY time(1h), "sensor_id"
END
```

In [ ]:
def create_continuous_aggregate(df, timestamp_col, group_cols, value_cols, freq):
    """
    Simulate creating a continuous aggregate (materialized view).
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input DataFrame
    timestamp_col : str
        Timestamp column name
    group_cols : list
        Columns to group by
    value_cols : list
        Columns to aggregate
    freq : str
        Time bucket frequency
    
    Returns:
    --------
    pd.DataFrame
        Aggregated DataFrame
    """
    df_copy = df.copy()
    df_copy['time_bucket'] = df_copy[timestamp_col].dt.floor(freq)
    
    agg_dict = {}
    for col in value_cols:
        agg_dict[col] = ['mean', 'min', 'max', 'std', 'count']
    
    result = df_copy.groupby(['time_bucket'] + group_cols).agg(agg_dict)
    result.columns = ['_'.join(col).strip() for col in result.columns.values]
    result = result.reset_index()
    
    return result

# Create hourly continuous aggregate
hourly_aggregate = create_continuous_aggregate(
    sensor_df,
    'timestamp',
    ['sensor_id'],
    ['temperature', 'humidity'],
    '1H'
)

print("Hourly Continuous Aggregate:")
hourly_aggregate

## Use Cases for Time Series Databases

### 1. Infrastructure Monitoring & Observability
- Server metrics (CPU, memory, disk, network)
- Application performance metrics (latency, throughput, error rates)
- Log aggregation and analysis
- Alerting on anomalies

### 2. Internet of Things (IoT)
- Sensor data collection (temperature, humidity, motion)
- Smart home/building automation
- Industrial equipment monitoring
- Predictive maintenance

### 3. Financial Data
- Stock prices and trading volumes
- Cryptocurrency markets
- Algorithmic trading signals
- Risk metrics and VaR calculations

### 4. Business Analytics
- Website traffic and user behavior
- Sales and revenue metrics
- Customer engagement tracking
- Real-time dashboards

In [ ]:
# Create a comprehensive dashboard showing all use cases
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'IoT: Sensor Temperature Over Time',
        'Monitoring: Server CPU by Host',
        'Financial: Stock Price Movement',
        'Analytics: Request Distribution'
    ),
    specs=[
        [{"type": "scatter"}, {"type": "bar"}],
        [{"type": "scatter"}, {"type": "histogram"}]
    ],
    vertical_spacing=0.12,
    horizontal_spacing=0.1
)

# IoT: Temperature trend
for i, sensor in enumerate(sensor_df['sensor_id'].unique()):
    sensor_data = sensor_df[sensor_df['sensor_id'] == sensor]
    # Sample to reduce points
    sampled = sensor_data.iloc[::10]
    fig.add_trace(
        go.Scatter(x=sampled['timestamp'], y=sampled['temperature'],
                   name=sensor, mode='lines'),
        row=1, col=1
    )

# Monitoring: Average CPU by host
cpu_by_host = server_df.groupby('host')['cpu_percent'].mean().sort_values(ascending=True)
fig.add_trace(
    go.Bar(x=cpu_by_host.values, y=cpu_by_host.index, orientation='h',
           marker_color=px.colors.sequential.Viridis,
           showlegend=False),
    row=1, col=2
)

# Financial: Close price
fig.add_trace(
    go.Scatter(x=financial_df['timestamp'], y=financial_df['close'],
               mode='lines', name='TECH', line=dict(color='green'),
               showlegend=False),
    row=2, col=1
)

# Analytics: Request count distribution
fig.add_trace(
    go.Histogram(x=server_df['request_count'], nbinsx=30,
                 marker_color='steelblue', showlegend=False),
    row=2, col=2
)

fig.update_layout(
    title='Time Series Database Use Cases Dashboard',
    height=700,
    template='plotly_white',
    showlegend=True
)

fig.update_xaxes(title_text='Time', row=1, col=1)
fig.update_xaxes(title_text='Avg CPU %', row=1, col=2)
fig.update_xaxes(title_text='Time', row=2, col=1)
fig.update_xaxes(title_text='Requests per Interval', row=2, col=2)

fig.update_yaxes(title_text='Temperature (°C)', row=1, col=1)
fig.update_yaxes(title_text='Host', row=1, col=2)
fig.update_yaxes(title_text='Price ($)', row=2, col=1)
fig.update_yaxes(title_text='Frequency', row=2, col=2)

fig

## Query Patterns for Time Series Databases

### Common Query Types

| Query Type | Description | Example |
|------------|-------------|----------|
| **Point query** | Get value at specific time | `WHERE time = '2024-01-15T10:00:00Z'` |
| **Range query** | Get values in time range | `WHERE time >= '2024-01-15' AND time < '2024-01-16'` |
| **Aggregation** | Compute statistics over time | `SELECT AVG(temp) GROUP BY time(1h)` |
| **Last value** | Get most recent value | `SELECT LAST(temp) FROM sensors` |
| **Top-N** | Get top values by metric | `ORDER BY cpu_percent DESC LIMIT 10` |
| **Gap filling** | Fill missing time periods | `time_bucket_gapfill('1 hour', time)` |

In [ ]:
# Demonstrate common time series query patterns with pandas

print("=== Common Time Series Query Patterns ===")

# 1. Point Query - Get value at specific time
specific_time = pd.Timestamp('2024-01-15 10:00:00')
point_query = sensor_df[
    (sensor_df['timestamp'] == specific_time) & 
    (sensor_df['sensor_id'] == 'sensor_1')
]
print(f"\n1. Point Query (at {specific_time}):")
print(point_query)

# 2. Range Query - Get values in time range
start_time = pd.Timestamp('2024-01-15 09:00:00')
end_time = pd.Timestamp('2024-01-15 10:00:00')
range_query = sensor_df[
    (sensor_df['timestamp'] >= start_time) & 
    (sensor_df['timestamp'] < end_time) &
    (sensor_df['sensor_id'] == 'sensor_1')
]
print(f"\n2. Range Query ({start_time} to {end_time}):")
print(f"   Records found: {len(range_query)}")
print(f"   Avg temperature: {range_query['temperature'].mean():.2f}°C")

# 3. Aggregation Query - Hourly averages
hourly_avg = sensor_df.set_index('timestamp').groupby(
    ['sensor_id', pd.Grouper(freq='1H')]
)['temperature'].mean().reset_index()
print(f"\n3. Aggregation Query (Hourly Averages):")
print(hourly_avg.head())

# 4. Last Value Query
last_values = sensor_df.groupby('sensor_id').apply(
    lambda x: x.nlargest(1, 'timestamp')
).reset_index(drop=True)[['sensor_id', 'timestamp', 'temperature']]
print(f"\n4. Last Value Query (Most Recent per Sensor):")
print(last_values)

# 5. Top-N Query - Highest CPU hosts
top_cpu = server_df.nlargest(5, 'cpu_percent')[['timestamp', 'host', 'cpu_percent']]
print(f"\n5. Top-N Query (Top 5 CPU Readings):")
print(top_cpu)

In [ ]:
# Gap Filling - Important for time series analysis
def gap_fill_time_series(df, timestamp_col, freq, value_col, method='ffill'):
    """
    Fill gaps in time series data with missing timestamps.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input DataFrame
    timestamp_col : str
        Timestamp column name
    freq : str
        Expected frequency
    value_col : str
        Value column to interpolate
    method : str
        Fill method: 'ffill', 'bfill', 'interpolate', 'zero'
    
    Returns:
    --------
    pd.DataFrame
        DataFrame with gaps filled
    """
    # Create complete time range
    full_range = pd.date_range(
        start=df[timestamp_col].min(),
        end=df[timestamp_col].max(),
        freq=freq
    )
    
    # Reindex to full range
    df_indexed = df.set_index(timestamp_col)
    df_reindexed = df_indexed.reindex(full_range)
    
    # Fill missing values
    if method == 'ffill':
        df_filled = df_reindexed.ffill()
    elif method == 'bfill':
        df_filled = df_reindexed.bfill()
    elif method == 'interpolate':
        df_filled = df_reindexed.interpolate(method='time')
    elif method == 'zero':
        df_filled = df_reindexed.fillna(0)
    
    return df_filled.reset_index().rename(columns={'index': timestamp_col})

# Simulate data with gaps by removing some records
sensor1 = sensor_df[sensor_df['sensor_id'] == 'sensor_1'].copy()
# Remove 10% of records randomly
mask = np.random.random(len(sensor1)) > 0.1
sensor1_with_gaps = sensor1[mask]

print(f"Original records: {len(sensor1)}")
print(f"Records with gaps: {len(sensor1_with_gaps)}")
print(f"Missing records: {len(sensor1) - len(sensor1_with_gaps)}")

# Fill gaps using interpolation
sensor1_filled = gap_fill_time_series(
    sensor1_with_gaps[['timestamp', 'temperature']],
    'timestamp', '1min', 'temperature', 'interpolate'
)

print(f"Records after gap fill: {len(sensor1_filled)}")

## Schema Design Best Practices

### InfluxDB Line Protocol
```
measurement,tag_key=tag_value field_key=field_value timestamp

# Example:
temperature,sensor_id=sensor_1,location=room_a value=22.5 1705312800000000000
```

### TimescaleDB Hypertables
```sql
-- Create regular table
CREATE TABLE sensor_data (
    time        TIMESTAMPTZ NOT NULL,
    sensor_id   TEXT NOT NULL,
    temperature DOUBLE PRECISION,
    humidity    DOUBLE PRECISION,
    pressure    DOUBLE PRECISION
);

-- Convert to hypertable (auto-partitioned by time)
SELECT create_hypertable('sensor_data', 'time');

-- Add space partitioning for high-cardinality data
SELECT create_hypertable('sensor_data', 'time',
    partitioning_column => 'sensor_id',
    number_partitions => 4);
```

### Design Guidelines

1. **Use appropriate tags/dimensions**: Low cardinality fields (sensor_id, region)
2. **Choose fields wisely**: High cardinality/numeric values as fields/measures
3. **Timestamp precision**: Match to your actual data resolution
4. **Chunk intervals**: Align with query patterns (1 hour, 1 day, 1 week)
5. **Index strategy**: Time + high-selectivity dimensions

## Performance Optimization Tips

### Write Optimization
- **Batch writes**: Group multiple data points into single write operations
- **Async writes**: Use asynchronous I/O for high-throughput ingestion
- **Buffer management**: Configure appropriate write buffers

### Query Optimization
- **Time-bound queries**: Always include time range predicates
- **Use continuous aggregates**: Pre-compute common aggregations
- **Limit result sets**: Use LIMIT and pagination
- **Index high-selectivity columns**: Reduce scan scope

### Storage Optimization
- **Enable compression**: Typically 10-20x storage reduction
- **Set retention policies**: Automatically remove old data
- **Use tiered storage**: Hot/warm/cold data tiers

In [ ]:
# Demonstrate batch write simulation
import time

def simulate_batch_writes(data, batch_size):
    """
    Simulate batched writes to a time series database.
    
    Parameters:
    -----------
    data : pd.DataFrame
        Data to write
    batch_size : int
        Number of records per batch
    
    Returns:
    --------
    dict
        Write statistics
    """
    total_records = len(data)
    num_batches = (total_records + batch_size - 1) // batch_size
    
    start_time = time.time()
    
    for i in range(0, total_records, batch_size):
        batch = data.iloc[i:i+batch_size]
        # Simulate write latency
        time.sleep(0.001)  # 1ms per batch
    
    elapsed = time.time() - start_time
    
    return {
        'total_records': total_records,
        'batch_size': batch_size,
        'num_batches': num_batches,
        'elapsed_seconds': elapsed,
        'records_per_second': total_records / elapsed
    }

# Compare different batch sizes
batch_sizes = [100, 500, 1000, 5000]
results = []

for batch_size in batch_sizes:
    stats = simulate_batch_writes(sensor_df, batch_size)
    results.append(stats)
    print(f"Batch size {batch_size:,}: {stats['num_batches']} batches, "
          f"{stats['records_per_second']:,.0f} records/sec")

# Visualize batch performance
results_df = pd.DataFrame(results)
fig = px.bar(
    results_df,
    x='batch_size',
    y='records_per_second',
    title='Write Throughput by Batch Size',
    labels={'batch_size': 'Batch Size', 'records_per_second': 'Records/Second'},
    template='plotly_white'
)
fig.update_layout(height=400)
fig

## 📌 Key Takeaways

### Time Series Databases Overview

| Aspect | Key Points |
|--------|------------|
| **Purpose** | Optimized storage and querying of time-stamped data |
| **Data Model** | Time-ordered, append-mostly, with tags and fields |
| **Key Features** | Compression, retention policies, downsampling, continuous aggregates |

### InfluxDB vs TimescaleDB

| Feature | InfluxDB | TimescaleDB |
|---------|----------|-------------|
| **Query Language** | Flux, InfluxQL | Full SQL |
| **Schema** | Schema-less | Schema-based (PostgreSQL) |
| **Best For** | Pure metrics/IoT | SQL + time series hybrid |
| **Ecosystem** | TICK stack | PostgreSQL ecosystem |

### Best Practices

1. **Retention Policies**: Define data lifecycle based on business requirements
2. **Downsampling**: Store high-resolution recent data, aggregate older data
3. **Compression**: Enable compression for 10-20x storage savings
4. **Continuous Aggregates**: Pre-compute common queries for faster dashboards
5. **Batch Writes**: Group writes for better throughput
6. **Time-Bounded Queries**: Always filter by time range

### Use Case Selection Guide

```
┌─────────────────────────────────────────────────────────────┐
│  Choose InfluxDB when:              Choose TimescaleDB when:│
│  • Pure metrics/monitoring          • Need full SQL         │
│  • DevOps/infrastructure            • Relational + TS data  │
│  • Rapid prototyping                • Complex analytics     │
│  • TICK stack integration           • PostgreSQL ecosystem  │
└─────────────────────────────────────────────────────────────┘
```

### Performance Considerations

- **Write throughput**: 100K-1M+ points/second achievable with proper batching
- **Query latency**: Sub-second for well-indexed recent data
- **Compression ratios**: 10-20x typical for sensor/metrics data
- **Retention trade-offs**: Balance storage costs vs. historical analysis needs